In [24]:
import pandas as pd

## Servizi
#### L'obiettivo di questa sezione è partire dal file che ci ha inviato AMA e ottenere il foglio "servizi" uguale a quello che avevamo ipotizzato in fake_data

In [25]:
df_services = pd.read_excel('Data/R12_Anagrafica_Servizi_Settimanale_PIPER.xlsx',sheet_name='Anagrafica Servizi Settimanale', skiprows=2)
df_barycentre = pd.read_excel('Data/R12_Anagrafica_Servizi_Settimanale_PIPER.xlsx',sheet_name='BARICENTRO',index_col=0, skiprows=0)
df_facilities = pd.read_excel('Data/R12_Anagrafica_Servizi_Settimanale_PIPER.xlsx',sheet_name='Impianti', skiprows=1)

In [26]:
days = ["LUN", "MAR", "MER", "GIO", "VEN", "SAB", "DOM"]
df_services[days]

,LUN,MAR,MER,GIO,VEN,SAB,DOM
0,P,P,P,P,P,P,NaN
1,P,P,P,P,P,P,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,M
3,N,N,N,N,N,N,N
4,P,P,P,P,P,P,NaN
...,...,...,...,...,...,...,...
815,NaN,NaN,N,NaN,NaN,NaN,NaN
816,NaN,N,NaN,NaN,N,NaN,NaN
817,P,NaN,NaN,P,NaN,NaN,NaN
818,P,NaN,NaN,P,NaN,NaN,NaN


In [27]:
print("shape: ", df_services.shape)
#df_servizi.head()

shape:  (820, 80)


In [28]:
df_vehicles_weight = pd.read_excel("Data/capacità veicoli per tipo servizio.xlsx", index_col=None, header=1).iloc[0:9,0:7].dropna(axis=1, how='all').fillna(0)
df_vehicles_weight.columns=['Etichette di riga', 'COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'COMP SIDE LOA 3A']
material_dict = {"A6 RACCOLTA CP CARTA": ["rifiuto rd carta", "COMP"],
                 "B2 RACCOLTA CP RESIDUO RU": ["rifiuto indifferenziato", "COMP"],
                 "A7 RACCOLTA CP MULTI": ["rifiuto rd multimateriale", "COMP"],
                 "C0 RACCOLTA CSL ORGANICO": ["rifiuto rd umido", "COMP SIDE LOA"],
                 "B4 RACCOLTA CSL CARTA": ["rifiuto rd carta", "COMP SIDE LOA"],
                 "B3 RACCOLTA CSL RESIDUO RU": ["rifiuto indifferenziato", "COMP SIDE LOA"],
                 "B9 RACCOLTA CSL MULTI": ["rifiuto rd multimateriale", "COMP SIDE LOA"]
                 }
df_vehicles_weight['Etichette di riga'].apply(lambda x: material_dict[x])
df_vehicles_weight['material'] = df_vehicles_weight['Etichette di riga'].apply(lambda x: material_dict[x][0])
df_vehicles_weight['vehicle'] = df_vehicles_weight['Etichette di riga'].apply(lambda x: material_dict[x][1])
df_vehicles_weight['2A'] = df_vehicles_weight['COMP 2A'] + df_vehicles_weight['COMP SIDE LOA 2A']
df_vehicles_weight['3A'] = df_vehicles_weight['COMP 3A'] + df_vehicles_weight['COMP SIDE LOA 3A']
df_vehicles_weight = df_vehicles_weight[['material', 'vehicle', '2A', '3A']]
df_vehicles_weight

,material,vehicle,2A,3A
0,rifiuto rd carta,COMP,2000.0,6000.0
1,rifiuto rd multimateriale,COMP,1500.0,4000.0
2,rifiuto indifferenziato,COMP,3000.0,10000.0
3,rifiuto indifferenziato,COMP SIDE LOA,5000.0,10000.0
4,rifiuto rd carta,COMP SIDE LOA,3000.0,6000.0
5,rifiuto rd multimateriale,COMP SIDE LOA,2000.0,4000.0
6,rifiuto rd umido,COMP SIDE LOA,5000.0,10000.0


In [29]:
vehicle_weight_dict = {key: {'dummy': 0} for key in df_vehicles_weight['material']}
for idx, mw in df_vehicles_weight.iterrows():
    vehicle_weight_dict[mw['material']].update({mw['vehicle'] + ' 2A': mw['2A']})
    vehicle_weight_dict[mw['material']].update({mw['vehicle'] + ' 3A': mw['3A']})
vehicle_weight_dict['rifiuto rd umido'].update({'COMP 2A': vehicle_weight_dict['rifiuto rd umido']['COMP SIDE LOA 2A']})
vehicle_weight_dict['rifiuto rd umido'].update({'COMP 3A': vehicle_weight_dict['rifiuto rd umido']['COMP SIDE LOA 3A']})
vehicle_weight_dict

{'rifiuto rd carta': {'dummy': 0,
  'COMP 2A': 2000.0,
  'COMP 3A': 6000.0,
  'COMP SIDE LOA 2A': 3000.0,
  'COMP SIDE LOA 3A': 6000.0},
 'rifiuto rd multimateriale': {'dummy': 0,
  'COMP 2A': 1500.0,
  'COMP 3A': 4000.0,
  'COMP SIDE LOA 2A': 2000.0,
  'COMP SIDE LOA 3A': 4000.0},
 'rifiuto indifferenziato': {'dummy': 0,
  'COMP 2A': 3000.0,
  'COMP 3A': 10000.0,
  'COMP SIDE LOA 2A': 5000.0,
  'COMP SIDE LOA 3A': 10000.0},
 'rifiuto rd umido': {'dummy': 0,
  'COMP SIDE LOA 2A': 5000.0,
  'COMP SIDE LOA 3A': 10000.0,
  'COMP 2A': 5000.0,
  'COMP 3A': 10000.0}}

In [30]:
#### questo dataframe ha tutte le righe di df_servizi, con aggiunte le lat e long del baricentro dell'itinerario.
df_services_barycenter = pd.merge(left = df_services, right = df_barycentre, left_on = "Itinerario cod", right_on = "Itinerario cod")
#df_services_barycenter.head()

In [31]:
df_services_plants = pd.merge(left = df_services_barycenter, right = df_facilities[["DESC_IMPIANTO", "SHAPE.MINY",	"SHAPE.MINX"]], left_on = "Destinazione Principale", right_on = "DESC_IMPIANTO")

In [32]:
dict_name_mapping = {"Itinerario cod": "service_id", "Latitudine": "lat","Longitudine": "long","SHAPE.MINY":  "lat_end", "SHAPE.MINX": "long_end", "TEMPO\n(H)":"time" , "Tipologia Rifiuto":"material", "PESO PRESUNTO\n(KG)": "quantity(kg)", "Tipo Mezzo Preferenziale": "AMA_vehicle", 'N°Contenitori\n(se raccolta)': "num_bins", "Destinazione Principale": "AMA_facility_desc", "CDL Gestore": "AMA_depot"}
df_service_internal = df_services_plants[["CDL Gestore", "Itinerario cod", "Latitudine", "Longitudine", "SHAPE.MINY","SHAPE.MINX", "TEMPO\n(H)" ,"Tipologia Rifiuto", "Tipo Mezzo Preferenziale", "PESO PRESUNTO\n(KG)", 'N°Contenitori\n(se raccolta)', "LUN", "MAR", "MER", "GIO", "VEN", "SAB", "DOM", "Destinazione Principale"]].rename(columns = dict_name_mapping)
### 2Assi, 2Assi voltabidoni, 2Assi voltasacchiebidoni - li possiamo trattare tutti allo stesso modo.
df_service_internal.replace({'COMP 2A V BID': 'COMP 2A', 'COMP 2A V SAC E B': 'COMP 2A'}, inplace = True)
set(df_service_internal["AMA_vehicle"])
df_service_internal = df_service_internal.set_index("service_id")
df_service_internal["material"] = df_service_internal["material"].str.lower()
df_service_internal.reset_index(inplace = True)
### trasformiamo il tempo del servizio da ore a minuti
df_service_internal["time"] *= 60
### we add the preferred_facility column to each service (both desc and id)
df_service_internal = pd.merge(left = df_service_internal, right = df_facilities[["COD_IMPIANTO", "DESC_IMPIANTO"]], left_on =  "AMA_facility_desc", right_on = "DESC_IMPIANTO").drop(columns = ['DESC_IMPIANTO']).rename(columns = {"COD_IMPIANTO": "AMA_facility_id"})
### aggiungiamo la capacità dei cassonetti a seconda del tipo di veicolo
mask_side_loader = (df_service_internal["AMA_vehicle"] == "COMP SIDE LOA 3A") |  (df_service_internal["AMA_vehicle"] == "COMP SIDE LOA 2A")
df_service_internal["bin_volume(m3)"] = 1.1

df_service_internal.loc[mask_side_loader, "bin_volume(m3)"] = 2.4
df_service_internal["volume(m3)"] = df_service_internal["bin_volume(m3)"]*df_service_internal["num_bins"]
df_service_internal

,service_id,AMA_depot,lat,long,lat_end,long_end,time,material,AMA_vehicle,quantity(kg),...,MAR,MER,GIO,VEN,SAB,DOM,AMA_facility_desc,AMA_facility_id,bin_volume(m3),volume(m3)
0,13B22P,AUTO_AC,41.755543,12.307232,41.850860,12.332095,49.333333,rifiuto indifferenziato,COMP 2A,1280,...,P,P,P,P,P,NaN,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,1.1,35.2
1,M12AR01,AUTO_AC,41.811028,12.477159,41.850860,12.332095,193.000000,rifiuto indifferenziato,COMP SIDE LOA 3A,6020,...,P,P,P,P,P,NaN,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,2.4,206.4
2,M12AR01M,AUTO_AC,41.811028,12.477159,41.850860,12.332095,190.166667,rifiuto indifferenziato,COMP SIDE LOA 3A,5950,...,NaN,NaN,NaN,NaN,NaN,M,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,2.4,204.0
3,M12AR01N,AUTO_AC,41.795607,12.461694,41.850860,12.332095,199.166667,rifiuto indifferenziato,COMP SIDE LOA 3A,6370,...,N,N,N,N,N,N,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,2.4,218.4
4,M12AS02,AUTO_AC,41.831535,12.529932,41.850860,12.332095,229.500000,rifiuto indifferenziato,COMP SIDE LOA 3A,7350,...,P,P,P,P,P,NaN,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,2.4,252.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,04RD10-CAB,AUTO_SA,41.969900,12.515066,41.816944,12.593219,278.000000,rifiuto rd carta,COMP SIDE LOA 3A,5460,...,NaN,NaN,M,NaN,NaN,NaN,ROMANA MACERI (RIFIUTO RD CARTA),IMP_000016_0013,2.4,218.4
815,04RD11-CAB,AUTO_SA,41.957313,12.517947,41.816944,12.593219,284.666667,rifiuto rd carta,COMP SIDE LOA 3A,5340,...,M,NaN,NaN,M,NaN,NaN,ROMANA MACERI (RIFIUTO RD CARTA),IMP_000016_0013,2.4,213.6
816,04RD12-CAB,AUTO_SA,41.967933,12.525606,41.816944,12.593219,306.666667,rifiuto rd carta,COMP SIDE LOA 3A,6240,...,NaN,M,NaN,NaN,M,NaN,ROMANA MACERI (RIFIUTO RD CARTA),IMP_000016_0013,2.4,249.6
817,20RD07-CA,AUTO_SA,42.001417,12.413598,41.816944,12.593219,222.000000,rifiuto rd carta,COMP SIDE LOA 3A,4020,...,NaN,M,NaN,NaN,M,NaN,ROMANA MACERI (RIFIUTO RD CARTA),IMP_000016_0013,2.4,160.8


In [33]:
df_service_internal["max_vehicle_cap(kg)"] = df_service_internal.apply(lambda x: vehicle_weight_dict[x['material']][x['AMA_vehicle']], axis=1)
df_service_internal["over_load"] = df_service_internal.apply(lambda x: "T" if x["quantity(kg)"] > x["max_vehicle_cap(kg)"] else "F", axis=1)
df_service_internal

,service_id,AMA_depot,lat,long,lat_end,long_end,time,material,AMA_vehicle,quantity(kg),...,GIO,VEN,SAB,DOM,AMA_facility_desc,AMA_facility_id,bin_volume(m3),volume(m3),max_vehicle_cap(kg),over_load
0,13B22P,AUTO_AC,41.755543,12.307232,41.850860,12.332095,49.333333,rifiuto indifferenziato,COMP 2A,1280,...,P,P,P,NaN,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,1.1,35.2,3000.0,F
1,M12AR01,AUTO_AC,41.811028,12.477159,41.850860,12.332095,193.000000,rifiuto indifferenziato,COMP SIDE LOA 3A,6020,...,P,P,P,NaN,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,2.4,206.4,10000.0,F
2,M12AR01M,AUTO_AC,41.811028,12.477159,41.850860,12.332095,190.166667,rifiuto indifferenziato,COMP SIDE LOA 3A,5950,...,NaN,NaN,NaN,M,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,2.4,204.0,10000.0,F
3,M12AR01N,AUTO_AC,41.795607,12.461694,41.850860,12.332095,199.166667,rifiuto indifferenziato,COMP SIDE LOA 3A,6370,...,N,N,N,N,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,2.4,218.4,10000.0,F
4,M12AS02,AUTO_AC,41.831535,12.529932,41.850860,12.332095,229.500000,rifiuto indifferenziato,COMP SIDE LOA 3A,7350,...,P,P,P,NaN,TMB MALAGROTTA 1/2 (RIFIUTO INDIFFERENZIATO),IMP_000010_0011,2.4,252.0,10000.0,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,04RD10-CAB,AUTO_SA,41.969900,12.515066,41.816944,12.593219,278.000000,rifiuto rd carta,COMP SIDE LOA 3A,5460,...,M,NaN,NaN,NaN,ROMANA MACERI (RIFIUTO RD CARTA),IMP_000016_0013,2.4,218.4,6000.0,F
815,04RD11-CAB,AUTO_SA,41.957313,12.517947,41.816944,12.593219,284.666667,rifiuto rd carta,COMP SIDE LOA 3A,5340,...,NaN,M,NaN,NaN,ROMANA MACERI (RIFIUTO RD CARTA),IMP_000016_0013,2.4,213.6,6000.0,F
816,04RD12-CAB,AUTO_SA,41.967933,12.525606,41.816944,12.593219,306.666667,rifiuto rd carta,COMP SIDE LOA 3A,6240,...,NaN,NaN,M,NaN,ROMANA MACERI (RIFIUTO RD CARTA),IMP_000016_0013,2.4,249.6,6000.0,T
817,20RD07-CA,AUTO_SA,42.001417,12.413598,41.816944,12.593219,222.000000,rifiuto rd carta,COMP SIDE LOA 3A,4020,...,NaN,NaN,M,NaN,ROMANA MACERI (RIFIUTO RD CARTA),IMP_000016_0013,2.4,160.8,6000.0,F


df_services_internal è il file dei servizi così come serve per far girare il modello di Gurobi.

Dubbi e domande:

 - quantity è la quantità in kili. Va bene?

## Depositi
#### L'obiettivo di questa sezione è partire dal file che ci ha inviato AMA e ottenere il foglio "depot" uguale a quello che avevamo ipotizzato in fake_data

In [34]:
df_cdl = pd.read_excel('Data/R12_Anagrafica_Servizi_Settimanale_PIPER.xlsx',sheet_name='CDL', skiprows=0)
depots = set(df_services["CDL Gestore"])
depots

{'AUTO_AC', 'AUTO_PM', 'AUTO_RC', 'AUTO_SA', 'AUTO_TP'}

In [35]:
#### RIGHE PER INSERIRE I VEICOLI IN DF_DEPOT

df_depot_internal = df_cdl[df_cdl["CDL"].isin(depots)].rename(columns = {"CDL": "depot_id"}).set_index("depot_id")[['Latitudine', 'Longitudine']]

dict_name_mapping = {"Latitudine": "lat", "Longitudine": "long"}
#df_disposal.columns
df_depot_internal = df_depot_internal.rename(columns = dict_name_mapping)
df_depot_internal

,lat,long
depot_id,,
AUTO_AC,41.769924,12.323795
AUTO_PM,41.840815,12.327650
AUTO_RC,41.884943,12.695035
AUTO_SA,41.968841,12.505559
AUTO_TP,41.801046,12.493676


In [36]:
df = pd.read_excel("Data\Copia di Parco mezzi autorimesse.xls", sheet_name = "dettaglio")
mask = (df["TIPOLOGIA INFOPMS"] != 'CM4') & (df["TIPOLOGIA INFOPMS"] != 'TS')
df = df.loc[mask,:]

dict_a = {"C2": 'COMP 2A',"C3": 'COMP 3A', "CSL2": 'COMP SIDE LOA 2A', "CSL3": 'COMP SIDE LOA 3A' }
df = df.replace({"TIPOLOGIA INFOPMS": dict_a})

dict2 = {"AUT_RC": "AUTO_RC", "AUT_PM": "AUTO_PM", "AUT_ROM": "AUTO_AC", "AUT_SA": "AUTO_SA", "AUT_TP": "AUTO_TP"}
df = df.replace({"UBICAZIONE": dict2})
groupby_dict = df.groupby(by = (["UBICAZIONE","TIPOLOGIA INFOPMS"]))["TARA"].count().unstack().to_dict(orient='index')

for key in groupby_dict.keys():
    df_depot_internal.loc[key, "vehicle"] = str(groupby_dict[key])

df_depot_internal

,lat,long,vehicle
depot_id,,,
AUTO_AC,41.769924,12.323795,"{'COMP 2A': 14, 'COMP 3A': 14, 'COMP SIDE LOA ..."
AUTO_PM,41.840815,12.327650,"{'COMP 2A': 35, 'COMP 3A': 16, 'COMP SIDE LOA ..."
AUTO_RC,41.884943,12.695035,"{'COMP 2A': 31, 'COMP 3A': 27, 'COMP SIDE LOA ..."
AUTO_SA,41.968841,12.505559,"{'COMP 2A': 32, 'COMP 3A': 21, 'COMP SIDE LOA ..."
AUTO_TP,41.801046,12.493676,"{'COMP 2A': 29, 'COMP 3A': 28, 'COMP SIDE LOA ..."


In [37]:
df_operators = pd.read_excel("Data/Ulteriori informazioni servizi svolti dagli autocentri.xlsx").iloc[49:54,1:4]
df_operators.columns = ["M", "P", "N"]
df_operators.index=depots
df_depot_internal = df_depot_internal.merge(df_operators, left_index=True, right_index=True)
df_depot_internal.index.name = "depot_id"
df_depot_internal

,lat,long,vehicle,M,P,N
depot_id,,,,,,
AUTO_AC,41.769924,12.323795,"{'COMP 2A': 14, 'COMP 3A': 14, 'COMP SIDE LOA ...",62,35,41
AUTO_PM,41.840815,12.327650,"{'COMP 2A': 35, 'COMP 3A': 16, 'COMP SIDE LOA ...",81,44,54
AUTO_RC,41.884943,12.695035,"{'COMP 2A': 31, 'COMP 3A': 27, 'COMP SIDE LOA ...",112,68,74
AUTO_SA,41.968841,12.505559,"{'COMP 2A': 32, 'COMP 3A': 21, 'COMP SIDE LOA ...",85,55,52
AUTO_TP,41.801046,12.493676,"{'COMP 2A': 29, 'COMP 3A': 28, 'COMP SIDE LOA ...",33,15,17


### colonne aggiunte per far girare il modello. Poi andranno eliminate

### colonne aggiunte per far girare il modello. Poi andranno eliminate

In [38]:
df_depot_internal["starting_t"] = "00:00:00"
df_depot_internal["ending_t"] = "23:59:00"
df_depot_internal["invest_cost"] = 1

df_depot_internal

,lat,long,vehicle,M,P,N,starting_t,ending_t,invest_cost
depot_id,,,,,,,,,
AUTO_AC,41.769924,12.323795,"{'COMP 2A': 14, 'COMP 3A': 14, 'COMP SIDE LOA ...",62,35,41,00:00:00,23:59:00,1
AUTO_PM,41.840815,12.327650,"{'COMP 2A': 35, 'COMP 3A': 16, 'COMP SIDE LOA ...",81,44,54,00:00:00,23:59:00,1
AUTO_RC,41.884943,12.695035,"{'COMP 2A': 31, 'COMP 3A': 27, 'COMP SIDE LOA ...",112,68,74,00:00:00,23:59:00,1
AUTO_SA,41.968841,12.505559,"{'COMP 2A': 32, 'COMP 3A': 21, 'COMP SIDE LOA ...",85,55,52,00:00:00,23:59:00,1
AUTO_TP,41.801046,12.493676,"{'COMP 2A': 29, 'COMP 3A': 28, 'COMP SIDE LOA ...",33,15,17,00:00:00,23:59:00,1


Le colonne "starting_t"	"ending_t"	"vehicle"	"invest_cost" mancano. Tempi di apertura e chiusura dell'impianto, tipo di veicoli e costo dell'impianto.

Dovrei aggiungere anche tutti gli altri CDL?

#### invest cost ci aspettiamo che arrivi


## Disposal
#### L'obiettivo di questa sezione è partire dal file che ci ha inviato AMA e ottenere il foglio "disposal" uguale a quello che avevamo ipotizzato in fake_data

In [39]:
df_disposal = pd.read_excel('Data/R12_Anagrafica_Servizi_Settimanale_PIPER.xlsx',sheet_name='Impianti', skiprows=1)

dict_name_mapping = {"COD_IMPIANTO": "disposal_id", "SHAPE.MINY": "lat","SHAPE.MINX": "long", "FRAZIONE":"material" , "Capacità (tonnellate / giorno)":"capacity(kg)", 'TEMPO DI STAZIONAMENTO IN IMPIANTO PER SCARICO\n (min)': "service_t"}

#df_disposal.columns
df_disposal = df_disposal[["COD_IMPIANTO","SHAPE.MINY", "SHAPE.MINX", "FRAZIONE", "Capacità (tonnellate / giorno)", 'TEMPO DI STAZIONAMENTO IN IMPIANTO PER SCARICO\n (min)', "DESC_IMPIANTO"]].rename(columns = dict_name_mapping)

### elimino quelle di trasferenza o trasbordo
filter1 = df_disposal["DESC_IMPIANTO"].str.contains('TRASFERENZA')
filter2 = df_disposal["DESC_IMPIANTO"].str.contains('TRASBORDO')
df_disposal_internal = df_disposal[~(filter1+filter2)].reset_index(drop = True).drop(columns = "DESC_IMPIANTO")
df_disposal_internal["material"] = df_disposal_internal["material"].str.lower()
df_disposal_internal = df_disposal_internal.set_index("disposal_id")
df_disposal_internal


,lat,long,material,capacity(kg),service_t
disposal_id,,,,,
IMP_000166_0014,41.802696,12.481962,rifiuto rd umido,NaN,20
IMP_000018_0013,41.709850,12.587212,rifiuto rd carta,NaN,20
IMP_000004_0011,41.885298,12.696229,rifiuto indifferenziato,NaN,20
IMP_000024_0014,41.898921,12.191789,rifiuto rd umido,NaN,20
IMP_000179_0013,41.782925,12.605017,rifiuto rd carta,NaN,20
IMP_999107_0011,41.701756,12.543336,rifiuto indifferenziato,NaN,20
IMP_000189_0011,41.769605,12.323630,rifiuto indifferenziato,NaN,20
IMP_000176_0013,41.685706,12.502530,rifiuto rd carta,NaN,20
IMP_000009_0012,41.675215,12.532817,rifiuto rd multimateriale,NaN,20


### colonne aggiunte per far girare il modello. Poi andranno eliminate

In [40]:
df_disposal_internal["starting_t"] = "00:00:00"
df_disposal_internal["ending_t"] = "23:59:00"
df_disposal_internal["invest_cost"] = 1

vehicle_list = ["COMP 2A", "COMP 3A", "COMP SIDE LOA 2A", "COMP SIDE LOA 3A"]   #lista dei veicoli che possono conferire al disposal
df_disposal_internal["vehicle"] = str(vehicle_list)
df_disposal_internal["capacity(kg)"] = 300*10**6
df_disposal_internal


,lat,long,material,capacity(kg),service_t,starting_t,ending_t,invest_cost,vehicle
disposal_id,,,,,,,,,
IMP_000166_0014,41.802696,12.481962,rifiuto rd umido,300000000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000018_0013,41.709850,12.587212,rifiuto rd carta,300000000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000004_0011,41.885298,12.696229,rifiuto indifferenziato,300000000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000024_0014,41.898921,12.191789,rifiuto rd umido,300000000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000179_0013,41.782925,12.605017,rifiuto rd carta,300000000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_999107_0011,41.701756,12.543336,rifiuto indifferenziato,300000000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000189_0011,41.769605,12.323630,rifiuto indifferenziato,300000000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000176_0013,41.685706,12.502530,rifiuto rd carta,300000000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000009_0012,41.675215,12.532817,rifiuto rd multimateriale,300000000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."


all'interno dei loro file mancano 
"starting_t", "ending_t", "invest_cost", "vehicle". 

E' giusto assumere che soltanto alcuni veicoli possano recarsi nei centri di smaltimento? Abbiamo bisogno di sapere per ogni impianto (normale o transhipment che sia) quali impianti possono andarci

"quantity" a oggi è nulla; è la capacità, giusto?

#### fondamentali per modello: quantity


## Transhipment 
#### L'obiettivo di questa sezione è partire dal file che ci ha inviato AMA e ottenere il foglio "transhipment" uguale a quello che avevamo ipotizzato in fake_data

In [41]:
df_transhipment_internal = df_disposal[(filter1+filter2)].reset_index(drop = True).drop(columns = "DESC_IMPIANTO").rename(columns = {"disposal_id": "transhipment_id"})
df_transhipment_internal["material"] = df_transhipment_internal["material"].str.lower()

df_transhipment_internal = df_transhipment_internal.set_index("transhipment_id")
df_transhipment_internal

,lat,long,material,capacity(kg),service_t
transhipment_id,,,,,
IMP_000051_0011,41.885298,12.696229,rifiuto indifferenziato,NaN,20
IMP_000222_0011,41.975417,12.496581,rifiuto indifferenziato,NaN,20
IMP_000178_0018,41.842214,12.330228,farmaci scaduti,NaN,20
IMP_000015_0012,41.842214,12.330228,rifiuto rd multimateriale,NaN,20


### colonne aggiunte per far girare il modello. Poi andranno eliminate

In [42]:
df_transhipment_internal["starting_t"] = "00:00:00"
df_transhipment_internal["ending_t"] = "23:59:00"
df_transhipment_internal["invest_cost"] = 1

vehicle_list = ["COMP 2A", "COMP 3A", "COMP SIDE LOA 2A", "COMP SIDE LOA 3A"]   #lista dei veicoli che possono conferire al disposal
df_transhipment_internal["vehicle"] = str(vehicle_list)

df_transhipment_internal["capacity(kg)"] = 100000
df_transhipment_internal

,lat,long,material,capacity(kg),service_t,starting_t,ending_t,invest_cost,vehicle
transhipment_id,,,,,,,,,
IMP_000051_0011,41.885298,12.696229,rifiuto indifferenziato,100000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000222_0011,41.975417,12.496581,rifiuto indifferenziato,100000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000178_0018,41.842214,12.330228,farmaci scaduti,100000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."
IMP_000015_0012,41.842214,12.330228,rifiuto rd multimateriale,100000,20,00:00:00,23:59:00,1,"['COMP 2A', 'COMP 3A', 'COMP SIDE LOA 2A', 'CO..."


**vedi cose mancanti sopra**

## VEHICLE

in particolare i campi *vehicle	type	depot	storage_cap	max_time	max_dist*

In [43]:
#df_vehicle = pd.read_excel('Data/Copia di Parco mezzi autorimesse.xls',sheet_name='Impianti', skiprows=1)
#df_vehicle_AMA = pd.read_excel("Data\Copia di Parco mezzi autorimesse.xls", sheet_name = "caratteristiche veicoli")


# df_vehicle["type"] = 

df_vehicle_internal = pd.DataFrame({"type": ["COMP 2A", "COMP 3A", "COMP SIDE LOA 2A", "COMP SIDE LOA 3A"]})
df_vehicle_internal["storage_cap_volume(m3)"] = [10, 22, 15,25]
df_vehicle_internal["storage_cap_weight(kg)"] = [3500, 12500, 4500,9000]
df_vehicle_internal["max_time"] = 480
df_vehicle_internal



,type,storage_cap_volume(m3),storage_cap_weight(kg),max_time
0,COMP 2A,10,3500,480
1,COMP 3A,22,12500,480
2,COMP SIDE LOA 2A,15,4500,480
3,COMP SIDE LOA 3A,25,9000,480


In [44]:
# dict_name_mapping = {"COD_IMPIANTO": "disposal_id", "SHAPE.MINY": "lat","SHAPE.MINX": "long", "FRAZIONE":"material" , "Capacità (tonnellate / giorno)":"quantity", 'TEMPO DI STAZIONAMENTO IN IMPIANTO PER SCARICO\n (min)': "service_t"} 

# #df_disposal.columns
# df_disposal = df_disposal[["COD_IMPIANTO","SHAPE.MINY", "SHAPE.MINX", "FRAZIONE", "Capacità (tonnellate / giorno)", 'TEMPO DI STAZIONAMENTO IN IMPIANTO PER SCARICO\n (min)', "DESC_IMPIANTO"]].rename(columns = dict_name_mapping)

# ### elimino quelle di trasferenza o trasbordo
# filter1 = df_disposal["DESC_IMPIANTO"].str.contains('TRASFERENZA')
# filter2 = df_disposal["DESC_IMPIANTO"].str.contains('TRASBORDO')
# df_disposal_internal = df_disposal[~(filter1+filter2)].reset_index(drop = True).drop(columns = "DESC_IMPIANTO")
# df_disposal_internal["material"] = df_disposal_internal["material"].str.lower()
# df_disposal_internal = df_disposal_internal.set_index("disposal_id")
# df_disposal_internal

In [45]:
df_facilities_internal = pd.concat((df_disposal_internal, df_transhipment_internal), axis = 0)  #concatenation of the two df
df_facilities_internal.index.names = ["facility_id"]


###  Save files

In [46]:
writer = pd.ExcelWriter('data/real_data.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_service_internal.to_excel(writer, sheet_name='Service')
df_depot_internal.to_excel(writer, sheet_name='Depot')
df_disposal_internal.to_excel(writer, sheet_name='Disposal')
df_transhipment_internal.to_excel(writer, sheet_name='Transship')
df_facilities_internal.to_excel(writer, sheet_name='Facility')
df_vehicle_internal.to_excel(writer, sheet_name = 'Vehicle')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

C:\Users\moden\AppData\Local\Temp\ipykernel_13136\3932720786.py:12: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


# Note:
dentro fake data, il foglio transhipment ha come index *transhipment*, mentre ora abbiamo *transhipment id*

## Create AMA capacity for the AMA-based scenario

In [157]:
df_real = pd.read_excel("Data/real_data.xlsx", sheet_name= "Service", index_col=0)
df_disposals = pd.read_excel("Data/real_data.xlsx", sheet_name= "Disposal", index_col=0)
df_transships = pd.read_excel("Data/real_data.xlsx", sheet_name= "Transship", index_col=0)
df_facilities = pd.concat([df_disposals,df_transships])
df_facilities.index.name = "facility_id"

In [158]:
dic_capacity = dict(zip(list(df_facilities.index), [0]*len(df_facilities.index) ))

In [159]:
for day_of_the_week in days:
    df_real_day = df_real[df_real[day_of_the_week].notna()]
    df_ama_solution = df_real_day[["service_id", "material","AMA_facility_id", "quantity(kg)"]].rename(columns = {"AMA_facility_id":"facility_id", "quantity(kg)":"service_quantity(kg)"})
    ## df_facility_results: kg and number of services per facility
    df_total_kg_per_facility = df_ama_solution.groupby("facility_id").agg(total_kg = ("service_quantity(kg)", "sum")).round(1)## Add 15% to the capacity and export dataframe
    for ind in df_total_kg_per_facility.index:
        dic_capacity[ind] = max(dic_capacity[ind], df_total_kg_per_facility.loc[ind, "total_kg"])
        
    

In [160]:
data = {'facility_id': dic_capacity.keys(), 'total_kg': dic_capacity.values()}
df_capacity = pd.DataFrame.from_dict(data).set_index("facility_id", drop = True)
df_capacity['T_or_D'] = 'D'
for ind in df_transships.index:
    df_capacity.loc[ind, "T_or_D"] = "T"

In [161]:
df_capacity["T_or_D"]=="D", "capacity"

(facility_id
 IMP_000166_0014       True
 IMP_000018_0013       True
 IMP_000004_0011       True
 IMP_000024_0014       True
 IMP_000179_0013       True
 IMP_999107_0011       True
 IMP_000189_0011       True
 IMP_000176_0013       True
 IMP_000009_0012       True
 IMP_000003_0012       True
 IMP_000050_0011       True
 IMP_000050_0011_1     True
 IMP_000017_0013       True
 IMP_000026_0020       True
 IMP_000016_0013       True
 IMP_147947_0017       True
 IMP_000010_0011       True
 IMP_000051_0011      False
 IMP_000222_0011      False
 IMP_000178_0018      False
 IMP_000015_0012      False
 Name: T_or_D, dtype: bool,
 'capacity')

In [162]:
# mean_capacity_T=df_capacity[df_capacity["T_or_D"]=="T"]["capacity"].max()
# mean_capacity_D=df_capacity[df_capacity["T_or_D"]=="D"]["capacity"].max()

# df_capacity.loc[(df_capacity['capacity']== 0) &(df_capacity['T_or_D']== "D"), "capacity"]= mean_capacity_D
# df_capacity.loc[(df_capacity['capacity']== 0) &(df_capacity['T_or_D']== "T"), "capacity"]= mean_capacity_T

In [163]:
df_capacity.to_excel("Data/df_AMA_capacity_max_day.xlsx", sheet_name='Capacity', engine='xlsxwriter')